In [31]:
import pytesseract
import re
import openpyxl as opxl
from PIL import Image, ImageDraw
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import re as regex
from collections import defaultdict
import cv2
import numpy as np

In [61]:
import cv2

refPt = []
coords = []
cropping = False

def click_and_crop(event, x, y, flags, param):
    global refPt, cropping, coords, image

    if event == cv2.EVENT_LBUTTONDOWN:
        refPt = [(x, y)]
        cropping = True

    elif event == cv2.EVENT_LBUTTONUP:
        refPt.append((x, y))
        cropping = False

        if len(refPt) == 2:
            x_start, y_start = refPt[0]
            x_end, y_end = refPt[1]
            width = x_end - x_start
            height = y_end - y_start
            bbox = (x_start, y_start, width, height)
            coords.append(bbox)

            cv2.rectangle(image, refPt[0], refPt[1], (0, 255, 0), 2)
            cv2.imshow("image", image)

        refPt = []

image = cv2.imread('sample2.jpg')
clone = image.copy()

cv2.namedWindow("image")
cv2.setMouseCallback("image", click_and_crop)

while True:
    cv2.imshow("image", image)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("r"):
        image = clone.copy()
        coords.clear()
    elif key == ord("q"):
        break

cv2.destroyAllWindows()

for i, box in enumerate(coords, 1):
    print(f"{i}: {box}")

1: (321, 40, 178, 82)


In [62]:
def string_conversion():
    pass


image = Image.open('sample2.jpg')
placeholders = ['', ' ']
for box_coords in coords:
    new_image = image.crop((box_coords[0], box_coords[1], box_coords[0] + box_coords[2], box_coords[3] + box_coords[1]))
    ocr_data = pytesseract.image_to_data(new_image, output_type=pytesseract.Output.DICT)
    print(ocr_data)
    ocr_data['text'] = [o for o in ocr_data['text'] if o not in placeholders]
    string = ' '.join(ocr_data['text'])
    print(ocr_data['text'])
    print(string)

{'level': [1, 2, 3, 4, 5, 5, 4, 5, 5, 3, 4, 5, 5, 3, 4, 5, 5, 5, 3, 4, 5, 5, 5, 5], 'page_num': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'block_num': [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'par_num': [0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4], 'line_num': [0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1], 'word_num': [0, 0, 0, 0, 1, 2, 0, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 3, 0, 0, 1, 2, 3, 4], 'left': [0, 7, 7, 7, 7, 55, 7, 7, 55, 7, 7, 7, 56, 7, 7, 7, 55, 82, 7, 7, 7, 32, 69, 103], 'top': [0, 7, 7, 7, 7, 7, 23, 23, 23, 38, 38, 38, 39, 54, 54, 54, 54, 42, 70, 70, 70, 71, 58, 70], 'width': [178, 156, 64, 59, 23, 11, 64, 32, 16, 86, 86, 25, 37, 107, 107, 41, 28, 32, 156, 156, 20, 31, 8, 60], 'height': [82, 70, 24, 7, 7, 7, 8, 8, 7, 8, 8, 8, 6, 9, 9, 7, 7, 28, 7, 7, 7, 6, 24, 7], 'conf': [-1, -1, -1, -1, 59, 55, -1, 0, 22, -1, -1, 37, 0, -1, -1, 35, 44, 33, -1, -1, 37, 30, 

In [3]:
#Method 2
image = cv2.imread('sample3.png')
grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(grayscale, (7, 7), 0)
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

In [10]:
# Load and preprocess
image = cv2.imread("sample3.png")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                               cv2.THRESH_BINARY_INV, 15, 10)

# Dilate to connect boxes
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
dilated = cv2.dilate(thresh, kernel, iterations=2)

# Find contours
contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
boxes = [cv2.boundingRect(c) for c in contours]

# Function to check if two boxes are near each other
def are_boxes_close(box1, box2, threshold=40):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Check if boxes are horizontally or vertically near
    horizontal_close = abs(x1 - x2) < threshold or abs((x1 + w1) - (x2 + w2)) < threshold
    vertical_close = abs(y1 - y2) < threshold or abs((y1 + h1) - (y2 + h2)) < threshold
    return horizontal_close and vertical_close

# Merge nearby boxes
merged = []
used = [False] * len(boxes)

for i in range(len(boxes)):
    if used[i]:
        continue
    x, y, w, h = boxes[i]
    for j in range(i + 1, len(boxes)):
        if used[j]:
            continue
        if are_boxes_close(boxes[i], boxes[j], threshold=25):
            x2, y2, w2, h2 = boxes[j]
            x = min(x, x2)
            y = min(y, y2)
            w = max(x + w, x2 + w2) - x
            h = max(y + h, y2 + h2) - y
            used[j] = True
    merged.append((x, y, w, h))

# Draw merged boxes
for (x, y, w, h) in merged:
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

cv2.imwrite('op.png', image)

True

In [13]:
#Optimised code @ 4/6/2025

def is_near(b1, b2, pad=15):
    x1, y1, w1, h1 = b1
    x2, y2, w2, h2 = b2

    # Create expanded bounding box of b1
    ex1 = x1 - pad
    ey1 = y1 - pad
    ex2 = x1 + w1 + pad
    ey2 = y1 + h1 + pad

    # Check if b2 is inside the expanded bounding box
    return (x2 >= ex1 and y2 >= ey1 and x2 + w2 <= ex2 and y2 + h2 <= ey2)

# Load and preprocess
image = cv2.imread("sample3.png")
sample_image = image.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                               cv2.THRESH_BINARY_INV, 15, 10)

# Strengthen contours
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
dilated = cv2.dilate(thresh, kernel, iterations=2)

# Detect contours
contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
boxes = [cv2.boundingRect(c) for c in contours]
boxes = sorted(boxes, key=lambda b: b[2]*b[3], reverse=True)

merged = []
used = [False] * len(boxes)
bbox_list = []
for i in range(len(boxes)):
    if used[i]:
        continue

    x1, y1, w1, h1 = boxes[i]
    max_x, max_y = x1 + w1, y1 + h1
    min_x, min_y = x1, y1

    for j in range(i + 1, len(boxes)):
        if used[j]:
            continue

        b2 = boxes[j]
        x2, y2, w2, h2 = b2
        x2b, y2b = x2 + w2, y2 + h2

        if is_near(boxes[i], b2):
            # Expand the larger box to include the smaller box
            min_x = min(min_x, x2)
            min_y = min(min_y, y2)
            max_x = max(max_x, x2b)
            max_y = max(max_y, y2b)
            used[j] = True

    used[i] = True
    merged.append((min_x, min_y, max_x - min_x, max_y - min_y))

# Draw result
for (x, y, w, h) in merged:
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    bbox_list.append((x, y, x+w, y+h))

cv2.imwrite('op.png', image)

True

In [14]:
print(bbox_list)

[(37, 3, 569, 44), (387, 45, 540, 126), (37, 245, 180, 324), (37, 139, 222, 189), (212, 76, 330, 123), (37, 208, 569, 218), (37, 128, 569, 138), (258, 252, 388, 291), (38, 46, 129, 93), (246, 219, 360, 251), (431, 139, 539, 168), (135, 52, 209, 83), (431, 176, 541, 196), (252, 296, 327, 324), (253, 45, 293, 75), (213, 45, 252, 74), (227, 252, 254, 277), (227, 296, 241, 324)]


In [44]:
image = Image.open('sample3.png')
list1 = []
for bbox in bbox_list:
    x1, y1, x2, y2 = bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1]+bbox[3]
    new_img = image.crop((x1, y1, x2, y2))
    ocr_data = pytesseract.image_to_string(new_img, output_type=pytesseract.Output.DICT)
    seperated_data = ocr_data['text'].split('\n')
    filtered_data = [s for s in seperated_data if ':' in s and 'Tests Requested' not in s]
    list1.append(filtered_data)

Dict = defaultdict(list)
empty_fields = ['', ' ']
print(list1)

for l in list1:
    for inst in l:
        print(inst)
        key, value = inst.split(':')
        if value in empty_fields:
            value = 'Not Available'
        Dict[key].append(value)
        
wb = opxl.load_workbook('output2.xlsx')
acs = wb.active
acs.append(list(Dict.keys()))
rows = list(zip(*Dict.values()))

for row in rows:
    acs.append(row)

wb.save('output2.xlsx')
wb.close()

[['Requisition Number: 760145584'], ['ACCOUNT DETAILS: Cash', 'Recept Number: Not Avalable', 'Member Number: Not Avaiale', 'Collected: 01/0272005', 'Received: 02/02/2005'], ['White Blood Calle: 7.49'], ['Location: PRETORIA', 'White Blood Calle: 7.49'], ['Member Number: Not Avaiale', 'Collected: 01/0272005', 'Received: 02/02/2005', 'Reported: 0702/2005'], ['White Blood Calle: 7.49 4-1 10°91.'], ['Collected: 01/0272005', 'Received: 02/02/2005', 'Reported: 0702/2005'], [], [], [], ['Collected: 01/02/2005', 'Received: 02/02/2005', 'Reported: 0702/2005'], ['DosJane Age: Hy'], ['Reported: 02/02/2005'], [], ['ACCOUNT DETAILS: Cash', 'Recept Number: Not Avalable'], ['ae Number:'], [], []]
Requisition Number: 760145584
ACCOUNT DETAILS: Cash
Recept Number: Not Avalable
Member Number: Not Avaiale
Collected: 01/0272005
Received: 02/02/2005
White Blood Calle: 7.49
Location: PRETORIA
White Blood Calle: 7.49
Member Number: Not Avaiale
Collected: 01/0272005
Received: 02/02/2005
Reported: 0702/2005
Whi